In [1]:
import random
import os
import time
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from tqdm import tqdm
from sklearn.model_selection import *
from transformers import *

In [2]:
CFG = {
    'fold_num': 5, 
    'seed': 42,
    'model': '../input/roberta-base',
    'max_len': 512,
    'epochs': 5,
    'train_bs': 24,
    'valid_bs': 32,
    'lr': 2e-5,
    'num_workers': 0,
    'weight_decay': 1e-6,
}

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['seed'])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
test_df = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')
test_df

,id,class,predictionstring
0,18409261F5C2,NaN,NaN
1,D46BCB48440A,NaN,NaN
2,0FB0700DAF44,NaN,NaN
3,D72CB1C11673,NaN,NaN
4,DF920E0A7337,NaN,NaN


In [5]:
test_names, test_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/test'))):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('../input/feedback-prize-2021/test/' + f, 'r').read())
test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
test_texts['text'] = test_texts['text'].apply(lambda x:x.split())
test_texts

100%|██████████| 5/5 [00:00<00:00, 366.16it/s]


,id,text
0,0FB0700DAF44,"[During, a, group, project,, have, you, ever, ..."
1,D72CB1C11673,"[Making, choices, in, life, can, be, very, dif..."
2,18409261F5C2,"[80%, of, Americans, believe, seeking, multipl..."
3,DF920E0A7337,"[Have, you, ever, asked, more, than, one, pers..."
4,D46BCB48440A,"[When, people, ask, for, advice,they, sometime..."


In [6]:
tokenizer = AutoTokenizer.from_pretrained(CFG['model'], add_prefix_space=True)

In [7]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        text = self.df.text.values[idx]
        
        return text

In [8]:
def collate_fn(data):
    input_ids, attention_mask = [], []
    
    tokenized_inputs = tokenizer(
        data,
        max_length=CFG['max_len'],
        padding='max_length',
        truncation=True,
        is_split_into_words=True,
        return_tensors='pt'
    )

    words = []
    for i in range(len(data)):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        words.append(word_ids)

    tokenized_inputs["word_ids"] = words
    
    return tokenized_inputs

In [9]:
test_loader = DataLoader(MyDataset(test_texts), batch_size=CFG['valid_bs'], collate_fn=collate_fn, shuffle=False, num_workers=4)
batch = next(iter(test_loader))
batch

{'input_ids': tensor([[    0,  1590,    10,  ...,  1843,  1901,     2],
        [    0, 11102,  5717,  ...,     1,     1,     1],
        [    0,  1812,   207,  ...,    33,    57,     2],
        [    0,  6319,    47,  ...,     6,    24,     2],
        [    0,   520,    82,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]]), 'word_ids': [[None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 15, 16, 16, 17, 18, 19, 20, 21, 22, 23, 24, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 40, 41, 41, 42, 43, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 64, 65, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 82, 82, 83, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 103, 103, 103, 104, 105, 10

In [10]:
model =  AutoModelForTokenClassification.from_pretrained(CFG['model'], num_labels=15).to(device)
model.load_state_dict(torch.load('../input/feedback-roberta/roberta-base_fold_0.pt', map_location=torch.device('cpu')))
model.eval()

Some weights of the model checkpoint at ../input/roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at ../input/roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stre

RobertaForTokenClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm

In [11]:
y_pred = []
words = []

with torch.no_grad():
    tk = tqdm(test_loader, total=len(test_loader), position=0, leave=True)
    for step, batch in enumerate(tk):
        word_ids = batch['word_ids']
        words.extend(word_ids)
        batch = {k: v.to(device) for k, v in batch.items() if k != 'word_ids'}

        output = model(**batch).logits

        y_pred.extend(output.argmax(-1).cpu().numpy())
        
y_pred = np.array(y_pred)

100%|██████████| 1/1 [00:13<00:00, 13.41s/it]


In [12]:
labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

In [13]:
final_preds = []

for i in tqdm(range(len(test_texts))):
    idx = test_texts.id.values[i]
    pred = ['']*len(test_texts.text.values[i])

    for j in range(len(y_pred[i])):
        if words[i][j] != None:
            pred[words[i][j]] = labels[y_pred[i][j]]

    pred = [x.replace('B-','').replace('I-','') for x in pred]

    preds = []
    j = 0
    while j < len(pred):
        cls = pred[j]
        if cls == 'O':
            j += 1
        end = j + 1
        while end < len(pred) and pred[end] == cls:
            end += 1
            
        if cls != 'O' and cls != '' and end - j > 10:
            final_preds.append((idx, cls, ' '.join(map(str, list(range(j, end))))))
        
        j = end
        
final_preds[0]

100%|██████████| 5/5 [00:00<00:00, 518.97it/s]


('0FB0700DAF44',
 'Lead',
 '0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40')

In [14]:
sub = pd.DataFrame(final_preds)
sub.columns = test_df.columns
sub

,id,class,predictionstring
0,0FB0700DAF44,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,0FB0700DAF44,Claim,49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64
2,0FB0700DAF44,Claim,66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 8...
3,0FB0700DAF44,Position,108 109 110 111 112 113 114 115 116 117 118 119
4,0FB0700DAF44,Evidence,123 124 125 126 127 128 129 130 131 132 133 13...
5,0FB0700DAF44,Claim,314 315 316 317 318 319 320 321 322 323 324 32...
6,0FB0700DAF44,Evidence,341 342 343 344 345 346 347 348 349 350 351 35...
7,D72CB1C11673,Lead,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
8,D72CB1C11673,Position,50 51 52 53 54 55 56 57 58 59 60
9,D72CB1C11673,Claim,63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 7...


In [15]:
sub.to_csv('submission.csv', index=False)